In [276]:
import re
import urllib.parse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
from urllib.parse import urlparse
import re

In [277]:
def process_csv(input_csv, output_csv):
    # Read the dataset
    df = pd.read_csv(input_csv)

    # Assuming the URLs are in a column named 'url'
    features_list = df['URL'].apply(extract_url_features).tolist()

    # Convert the list of feature dictionaries to a DataFrame
    features_df = pd.DataFrame(features_list)

    # Concatenate the original DataFrame with the features DataFrame
    result_df = pd.concat([df, features_df], axis=1)

    # Save the result to a new CSV file
    result_df.to_csv(output_csv, index=False)

In [278]:
input_csv = 'dataset_url.csv'  # Replace with your input CSV file path
output_csv = 'output_features.csv'  # Replace with your desired output CSV file path
process_csv(input_csv, output_csv)

In [279]:
df='output_features.csv'

In [280]:
df = pd.read_csv(df)

In [281]:
df

,Unnamed: 0,URL,status,scheme,count_dots,count_slashes,count_special_chars,url_length,domain,num_digits,num_links
0,0,http://www.conceptdraw.com/How-To-Guide/Local-...,0,http,2,4,11,58,www.conceptdraw.com,0,0
1,1,https://www.edrawsoft.com/Local-Area-Network.php,0,https,3,3,9,48,www.edrawsoft.com,0,0
2,2,http://www.webopedia.com/TERM/L/local_area_net...,0,http,2,5,11,54,www.webopedia.com,0,0
3,3,https://www.acsac.org/secshelf/book001/16.pdf,0,https,3,5,9,45,www.acsac.org,5,0
4,4,http://www.diffen.com/difference/LAN_vs_WAN,0,http,2,4,9,43,www.diffen.com,0,0
...,...,...,...,...,...,...,...,...,...,...,...
73570,73570,http://www.grupolyl.com/slap/GD/,1,http,2,5,8,32,www.grupolyl.com,0,0
73571,73571,http://bcgroup-sa.com/nilalalaa/GD/index.php,1,http,2,5,9,44,bcgroup-sa.com,0,0
73572,73572,http://mainepta.org/eme/gm/excel/excel/excel.p...,1,http,16,7,53,286,mainepta.org,62,6
73573,73573,http://support.escola-olivera.org/GOOGLENEWW/G...,1,http,2,10,14,95,support.escola-olivera.org,0,0


In [282]:
df.columns

Index(['Unnamed: 0', 'URL', 'status', 'scheme', 'count_dots', 'count_slashes',
       'count_special_chars', 'url_length', 'domain', 'num_digits',
       'num_links'],
      dtype='object')

In [283]:
df = df.drop(columns=['Unnamed: 0', 'domain','URL'])

In [285]:
df.columns

Index(['status', 'scheme', 'count_dots', 'count_slashes',
       'count_special_chars', 'url_length', 'num_digits', 'num_links'],
      dtype='object')

In [286]:
# Separate features and target
X = df.drop(columns=['status'])  # Features
y = df['status'] 

In [287]:
X = pd.get_dummies(X)

In [288]:
X.columns

Index(['count_dots', 'count_slashes', 'count_special_chars', 'url_length',
       'num_digits', 'num_links', 'scheme_ftp', 'scheme_http', 'scheme_https'],
      dtype='object')

In [290]:
y

0        0
1        0
2        0
3        0
4        0
        ..
73570    1
73571    1
73572    1
73573    1
73574    1
Name: status, Length: 73575, dtype: int64

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [292]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [293]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.89
              precision    recall  f1-score   support

           0       0.90      0.87      0.89     10966
           1       0.88      0.91      0.89     11107

    accuracy                           0.89     22073
   macro avg       0.89      0.89      0.89     22073
weighted avg       0.89      0.89      0.89     22073



In [294]:
X.columns

Index(['count_dots', 'count_slashes', 'count_special_chars', 'url_length',
       'num_digits', 'num_links', 'scheme_ftp', 'scheme_http', 'scheme_https'],
      dtype='object')

In [296]:
# Function to extract features from the URL
def extract_features(url):
    count_dots = url.count('.')
    count_slashes = url.count('/')
    count_special_chars = len(re.findall(r'[^a-zA-Z0-9]', url)) - count_dots - count_slashes
    url_length = len(url)
    num_digits = len(re.findall(r'\d', url))
    num_links = len(re.findall(r'https?://', url))
    scheme_ftp = int(urlparse(url).scheme == 'ftp')
    scheme_http = int(urlparse(url).scheme == 'http')
    scheme_https = int(urlparse(url).scheme == 'https')
    
    return [count_dots, count_slashes, count_special_chars, url_length, num_digits, num_links, scheme_ftp, scheme_http, scheme_https]

In [327]:
def get_prediction_from_url(test_url):
    features_test = extract_features(test_url)
    # Due to updates to scikit-learn, we now need a 2D array as a parameter to the predict function.
    features_test = np.array(features_test).reshape((1, -1))
    
    return features_test

In [357]:
url = 'http://www.twitter.com'

list = get_prediction_from_url(url)

In [368]:
# Make predictions using the loaded model
new_predictions = loaded_model.predict(list)
print(new_predictions[0])

1


C:\Users\madhu\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
